In [1]:
from glob import glob
import pandas as pd
from datasets import Dataset, load_from_disk
from datasets import DatasetDict
import datasets

import pickle 

/shared/miniconda3/envs/peerat_local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "/home/aisg/peerat/Thai_local_benchmark/datasets/*.csv"

In [3]:
food_name = {
    "north": ["ลาบเหนือ",
            "ขนมจีนน้ำเงี้ยว",
            "แกงฮังเล",
            "น้ำพริกหนุ่ม",
            "น้ำพริกอ่อง",
            "แกงหัวปลี",
            "แกงหยวก",
            "ไส้อั๋ว",
            "แคบหมู",
            "แกงกระด้าง"],
    "east": ["แจ่วบักเขือ",
            "อ่อมน้องวัว",
            "ตำบักหุ่ง",
            "หมกหน่อไม้",
            "ซอยจุ๊",
            "แกงขี้เหล็ก",
            "น้ำตกคอหมูย่าง",
            "ปลาส้ม",
            "แจ่วฮ้อน",
            "ตับหวาน"],
    "south": ["แก๋งพุงปลา",
            "แกงส้ม",
            "ลอกอผัดไข่",
            "ข้าวผั๊ดเคย",
            "แกงส้มปล๊าพงยอดพร๊าว",
            "ข้าวผั๊ดเคย",
            "ปล่าซายท๋อดขี้มิ้น",
            "ผั๊ดผั๊กเหลี๊ยง",
            "แก๋งเลียง",
            "ผั๊ดหลูกต๋อ"],
    "center": ["ผัดกระเพราหมูสับไข่ดาว", "ต้มยำกุ้ง", "ผัดไทกุ้งสด", "แกงเขียวหวานไก่", "ต้มข่าไก่",
                "ต้มจืด",
                "พะโล้",
                "ทอดมันปลากราย",
                "เต้าเจี้ยวหลน",
                "พะแนง"]
}

conversation_list = [
    "ประเพณีท้องถิ่น",
    "อาชีพ",
    "การคมนาคม",
    "ครอบครัว",
    "กลับบ้านช่วงเทศกาล",
    "การเมืองท้องถิ่น",
    "สภาพอากาศ",
    "อาหารท้องถิ่น",
    "เครื่องแต่งกายท้องถิ่น",
    "การศึกษาท้องถิ่น",
]

In [18]:
local_data = DatasetDict()

for local in ["east","north","south","center"]:
    task_data = DatasetDict()
    for file in glob(path):
        task_name = file.split("/")[-1].split(".")[0]
        df = pd.read_csv(file)
        if task_name == 'qa':
            new_df = df[["id","question",'text_2','context']]
            dataset = Dataset.from_pandas(new_df)
            task_data[task_name] = dataset
        elif task_name == 'summarization':
            new_df = df[["id",f"text_2_{local}","text_1"]]
            dataset = Dataset.from_pandas(new_df)
            dataset = dataset.rename_column(f"text_2_{local}", "text_2")
            task_data[task_name] = dataset
        elif 'translation' in task_name:
            task_name = f'translation_eng_{local}'
            new_df = df[["english",local]]
            dataset = Dataset.from_pandas(new_df)
            dataset = dataset.rename_column(f"english", "text_1")
            dataset = dataset.rename_column(local, "text_2")
            task_data[task_name] = dataset

            new_df = df[["english",local]]
            dataset = Dataset.from_pandas(new_df)
            dataset = dataset.rename_column(f"english", "text_2")
            dataset = dataset.rename_column(local, "text_1")
            task_data[task_name] = dataset


    # conversation
    task_name='conversation'
    new_df = pd.DataFrame(list(zip(list(range(len(conversation_list))), conversation_list, "."*len(conversation_list))),columns =['id', 'text_1',"text_2"])
    dataset = Dataset.from_pandas(new_df)
    task_data[task_name] = dataset

    task_name='food'
    food_list = food_name[local]
    new_df = pd.DataFrame(list(zip(list(range(len(food_list))), food_list, "."*len(food_list))),columns =['id', 'text_1', "text_2"])
    dataset = Dataset.from_pandas(new_df)
    task_data[task_name] = dataset

    local_data[local] = task_data

In [19]:
with open('datasets/dataset.pkl', 'wb') as file:  
    # A new file will be created 
    pickle.dump(local_data, file) 

In [20]:
with open('datasets/dataset.pkl', 'rb') as file: 
      
    # Call load method to deserialze 
    loaded_file = pickle.load(file) 
  
    print(loaded_file) 

DatasetDict({
    east: DatasetDict({
        qa: Dataset({
            features: ['id', 'question', 'text_2', 'context'],
            num_rows: 20
        })
        summarization: Dataset({
            features: ['id', 'text_2', 'text_1'],
            num_rows: 20
        })
        translation_eng_east: Dataset({
            features: ['text_2', 'text_1'],
            num_rows: 20
        })
        conversation: Dataset({
            features: ['id', 'text_1', 'text_2'],
            num_rows: 10
        })
        food: Dataset({
            features: ['id', 'text_1', 'text_2'],
            num_rows: 10
        })
    })
    north: DatasetDict({
        qa: Dataset({
            features: ['id', 'question', 'text_2', 'context'],
            num_rows: 20
        })
        summarization: Dataset({
            features: ['id', 'text_2', 'text_1'],
            num_rows: 20
        })
        translation_eng_north: Dataset({
            features: ['text_2', 'text_1'],
            num_r

In [25]:
loaded_file['north']['summarization'][1]

{'id': 'features-39791695',
 'text_2': 'ผลการศึกษาล่าสุดของมหาลัยเซนต์แอนดรูส์ในสหราชอาณาจักรพบว่า แมวน้ำตี้อาศัยตามเส้นตางเดินเรือตี้มีการสัญจรไปมาคับคั่ง อาจได้ฮับผลกระทบจากมลภาวะตางเสียงจนยะฮื้สูญเสียก๋านได้ยินเป็นชั่วคราวได้',
 'text_1': 'เสียงดังในเส้นทางเดินเรือที่มีการสัญจรไปมาคับคั่ง ทำให้แมวน้ำและสัตว์อื่น ๆ สูญเสียการได้ยินชั่วคราวได้ มีการเผยแพร่ผลการศึกษาดังกล่าวในวารสาร Applied Ecology โดยดร. เอสเทอร์ โจนส์ นักนิเวศวิทยาผู้ศึกษาเรื่องนี้ระบุว่า สัตว์น้ำที่อาศัยอยู่ในบริเวณเส้นทางเดินเรือสินค้าที่มีการสัญจรไปมาหนาแน่น โดยเฉพาะอย่างยิ่งแมวน้ำสีเทา และแมวน้ำที่อาศัยอยู่ตามริมอ่าว (Harbour seal) ก็เหมือนกับมนุษย์ที่อาศัยอยู่ในเมืองใหญ่ที่มีเสียงรบกวนดังตลอดเวลา มลภาวะทางเสียงจากเรือเดินสมุทร นอกจากจะเป็นอุปสรรคในการหาอาหารและการสื่อสารกันในฝูงแมวน้ำแล้ว ยังทำให้แมวน้ำเกิดอาการหูดับขึ้นชั่วคราวได้ เมื่อมีเสียงรบกวนในระดับที่ดังเกินมาตรฐาน ซึ่งขณะนี้มีแหล่งที่อยู่อาศัยของแมวน้ำในสหราชอาณาจักรที่ทับซ้อนกับเส้นทางเดินเรือถึง 11 แห่ง จากทั้งหมด 25 แห่ง สหราชอาณาจักรมีเส้นทางเดินเรื